In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from ours_eval import csls_knn_10_score, evaluation, separate_eva
from icp import ICPTrainer
import matplotlib.pyplot as plt
import utils
from utils import sub_icp, find_clts, find_centers, csls_knn_10_score
import params
import sklearn.cluster
from sklearn.decomposition import PCA

In [2]:
def normalize(x):
    return x / np.linalg.norm(x, ord=2, axis=1, keepdims=True)

In [3]:
src_id2word, src_word2id, src_embeddings = utils.read_txt_embeddings('data/wiki.%s.vec' % params.src_lang, params.n_eval_ex, False) #n_eval_ex = 200000
tgt_id2word, tgt_word2id, tgt_embeddings = utils.read_txt_embeddings('data/wiki.%s.vec' % params.tgt_lang, params.n_eval_ex, False)
src_normed = normalize(src_embeddings)
tgt_normed = normalize(tgt_embeddings)
cross_dict_src2tgt = utils.load_dictionary('data/%s-%s.5000-6500.txt' % (params.src_lang, params.tgt_lang), src_word2id, tgt_word2id)
cross_dict_tgt2src = utils.load_dictionary('data/%s-%s.5000-6500.txt' % (params.tgt_lang, params.src_lang), tgt_word2id, src_word2id)

Loaded 200000 pre-trained word embeddings.
Loaded 200000 pre-trained word embeddings.
data/en-es.5000-6500.txt
Found 2975 pairs of words in the dictionary (1500 unique). 0 other pairs contained at least one unknown word (0 in lang1, 0 in lang2)
data/es-en.5000-6500.txt
Found 2416 pairs of words in the dictionary (1500 unique). 0 other pairs contained at least one unknown word (0 in lang1, 0 in lang2)


In [4]:
T = np.load("%s/%s_%s_T.npy" % (params.cp_dir, params.src_lang, params.tgt_lang))
T2 = np.load("%s/%s_%s_T.npy" % (params.cp_dir, params.tgt_lang, params.src_lang))
TranslatedX = src_embeddings.dot(np.transpose(T))

In [5]:
src_full = np.load("data/%s_%d.npy" % (params.src_lang, params.n_init_ex)) # 10000, 10000 english
src_trans = src_full.dot(np.transpose(T))
tgt_full = np.load("data/%s_%d.npy" % (params.tgt_lang, params.n_init_ex)) # 300, 10000 es

# for i in range(5):
src_trans_normed = normalize(src_trans)
tgt_full_normed = normalize(tgt_full)
'''
pca_25 = PCA(n_components = 25)
pca_25.fit(src_full_normed)
src_W = pca_25.transform(src_full_normed)
pca_25.fit(tgt_full_normed)
tgt_W = pca_25.transform(tgt_full_normed)
'''

'\npca_25 = PCA(n_components = 25)\npca_25.fit(src_full_normed)\nsrc_W = pca_25.transform(src_full_normed)\npca_25.fit(tgt_full_normed)\ntgt_W = pca_25.transform(tgt_full_normed)\n'

In [6]:
n_clusters = 2

src_clt = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init= 40, random_state=200)
'''
tgt_clt= sklearn.cluster.KMeans(n_clusters=n_clusters, n_init= 40, random_state=200)
src_clt = sklearn.cluster.SpectralClustering(n_clusters = n_clusters, n_init= 40, random_state=200)
'''
src_y = src_clt.fit_predict(src_trans_normed)
tgt_y = src_clt.predict(tgt_full_normed)

In [7]:
src_centers, tgt_centers, src_dic, tgt_dic = find_centers(src_full, tgt_full, src_y, tgt_y, n_clts = n_clusters)
Translated_centers = src_centers.dot(np.transpose(T))

[(0, 2940), (1, 2060)]
[(0, 3494), (1, 1506)]


In [8]:
trans_c = normalize(Translated_centers)
tgt_c = normalize(tgt_centers)
#trans_c = Translated_centers
#tgt_c = tgt

cos_dist = trans_c.dot(tgt_c.T)
print(cos_dist)

[[0.93892501 0.76449506]
 [0.77669488 0.95015175]]


In [9]:
mk_dic = {0:3, 1:1, 2:0, 3:2, 4:4}

In [10]:
cross_dict_src2tgt = utils.load_dictionary('data/%s-%s.5000-6500.txt' % (params.src_lang, params.tgt_lang), src_word2id, tgt_word2id)
cross_dict_tgt2src = utils.load_dictionary('data/%s-%s.5000-6500.txt' % (params.tgt_lang, params.src_lang), tgt_word2id, src_word2id)

data/en-es.5000-6500.txt
Found 2975 pairs of words in the dictionary (1500 unique). 0 other pairs contained at least one unknown word (0 in lang1, 0 in lang2)
data/es-en.5000-6500.txt
Found 2416 pairs of words in the dictionary (1500 unique). 0 other pairs contained at least one unknown word (0 in lang1, 0 in lang2)


In [11]:
src_classes = find_clts(data = src_embeddings, centers = src_centers, dico = cross_dict_src2tgt)
tgt_classes = find_clts(data = tgt_embeddings, centers = tgt_centers, dico = cross_dict_tgt2src)

src_classes_trans = find_clts(data = src_embeddings, centers = src_centers, dico = cross_dict_src2tgt, trans = True)
src_correct = np.where(src_classes == src_classes_trans)
src_acc = src_correct[0].shape[0]/1500
tgt_classes_trans = find_clts(data = tgt_embeddings, centers = tgt_centers, dico = cross_dict_tgt2src, trans = True)
tgt_correct = np.where(tgt_classes == tgt_classes_trans)
tgt_acc = tgt_correct[0].shape[0]/1500
print(src_acc, tgt_acc)

1.002 0.824


In [12]:
TX, TY = utils.multi_ICP(src_full, tgt_full, src_y, tgt_y, src_dic, tgt_dic, n_clusters, time_run_icp = 5)

cluster 0 src shape: (2940, 300) tgt shape: (3494, 300)
0: Rec 6.799466 BB 551 Time: 15.941349


In [18]:
result_src = separate_eva(src_embeddings, tgt_embeddings, T, TX, src_classes, dico=cross_dict_src2tgt)
result_tgt = separate_eva(tgt_embeddings, src_embeddings, T2, TY, tgt_classes, dico=cross_dict_tgt2src)

953 source words -   - Precision at k = 1: 0.000000
953 source words -   - Precision at k = 5: 0.209864
953 source words -   - Precision at k = 10: 0.209864
547 source words -   - Precision at k = 1: 24.497258
547 source words -   - Precision at k = 5: 44.058501
547 source words -   - Precision at k = 10: 55.027422
2447 source words -   - Precision at k = 1: 10.391563
2447 source words -   - Precision at k = 5: 18.810144
2447 source words -   - Precision at k = 10: 23.463082
936 source words -   - Precision at k = 1: 0.000000
936 source words -   - Precision at k = 5: 0.106838
936 source words -   - Precision at k = 10: 0.106838
564 source words -   - Precision at k = 1: 35.638298
564 source words -   - Precision at k = 5: 56.560284
564 source words -   - Precision at k = 10: 61.879433
2069 source words -   - Precision at k = 1: 13.917711
2069 source words -   - Precision at k = 5: 22.153423
2069 source words -   - Precision at k = 10: 24.230693
